In [ ]:
import os
import re
import sys
import tempfile
import subprocess
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import html2text
from pdfminer.high_level import extract_text
from docx import Document
import pypandoc
import logging
from concurrent.futures import ThreadPoolExecutor

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("conversion.log"), logging.StreamHandler()],
)


class DocumentConverter:
    def __init__(self):
        # 永中Office路径配置（根据实际安装位置修改）
        self.yozo_path = {
            "cmd": r"C:\Program Files (x86)\Yozosoft\Yozo_Office\Jre\bin\java.exe",
            "jar": r"C:\Program Files (x86)\Yozosoft\Yozo_Office\Yozo_Office.jar",
        }
        self.supported_ext = [".pdf", ".doc", ".docx", ".html", ".htm", ".xls", ".xlsx"]

    def sanitize_filename(self, filename):
        """清理文件名中的非法字符"""
        return re.sub(r'[\\/*?:"<>|]', "", filename).strip()

    def convert_pdf(self, pdf_path):
        """转换PDF到Markdown"""
        try:
            text = extract_text(pdf_path)
            # 后处理：保留基本结构
            text = re.sub(r"\n{3,}", "\n\n", text)
            return text
        except Exception as e:
            raise RuntimeError(f"PDF转换失败: {str(e)}")

    def convert_docx(self, docx_path):
        """转换DOCX到Markdown（处理表格和标题）"""
        try:
            doc = Document(docx_path)
            output = []

            for element in doc.element.body.iterchildren():
                if isinstance(element, CT_P):
                    para = Paragraph(element, doc)
                    if para.text.strip():
                        if para.style.name.startswith("Heading"):
                            level = min(int(para.style.name[-1]), 6)
                            output.append(f"{'#' * level} {para.text}")
                        else:
                            output.append(para.text)
                elif isinstance(element, CT_Tbl):
                    table = Table(element, doc)
                    table_data = [
                        [cell.text.strip() for cell in row.cells] for row in table.rows
                    ]
                    table_md = (
                        "\n[表格开始]\n"
                        + "\n".join("| " + " | ".join(row) + " |" for row in table_data)
                        + "\n[表格结束]\n"
                    )
                    output.append(table_md)

            return "\n\n".join(output)
        except Exception as e:
            raise RuntimeError(f"DOCX转换失败: {str(e)}")

    def convert_html(self, html_path):
        """转换HTML到Markdown（处理表单）"""
        try:
            with open(html_path, "r", encoding="utf-8") as f:
                html = f.read()

            # 预处理表单
            html = re.sub(r"<form\b[^>]*>", "\n[表单开始]\n", html)
            html = re.sub(r"</form>", "\n[表单结束]\n", html)

            h = html2text.HTML2Text()
            h.ignore_links = False
            h.bypass_tables = False
            h.ignore_images = True
            h.body_width = 0

            md = h.handle(html)
            # 后处理
            md = re.sub(r"(\n\s*){3,}", "\n\n", md)
            return md
        except Exception as e:
            raise RuntimeError(f"HTML转换失败: {str(e)}")

    def convert_excel(self, excel_path):
        """转换Excel到Markdown表格"""
        try:
            xl = pd.ExcelFile(excel_path)
            output = []

            for sheet_name in xl.sheet_names:
                df = xl.parse(sheet_name, header=None, dtype=str)
                df = df.fillna("")

                table_md = f"\n# {sheet_name}\n\n[表格开始]\n"
                for _, row in df.iterrows():
                    if not row.str.strip().eq("").all():
                        table_md += (
                            "| " + " | ".join(row.astype(str).str.strip()) + " |\n"
                        )
                table_md += "[表格结束]\n"
                output.append(table_md)

            return "\n".join(output)
        except Exception as e:
            raise RuntimeError(f"Excel转换失败: {str(e)}")

    def convert_doc_with_yozo(self, doc_path):
        """添加JVM参数绕过验证"""
        try:
            temp_dir = Path(tempfile.mkdtemp())
            docx_path = temp_dir / f"{Path(doc_path).stem}.docx"

            cmd = [
                self.yozo_path["cmd"],
                "-XX:-UseSplitVerifier",  # 禁用严格验证
                "-noverify",  # 跳过字节码验证
                "-jar",
                self.yozo_path["jar"],
                "-convert",
                str(doc_path),
                str(docx_path),
                "-format",
                "docx",
            ]

            subprocess.run(cmd, check=True, stderr=subprocess.PIPE)

            # 转换DOCX到MD
            md_content = self.convert_docx(docx_path)
            return md_content
        except subprocess.CalledProcessError as e:
            error_msg = e.stderr.decode("gbk", errors="ignore")
            raise RuntimeError(f"永中转换失败: {error_msg}")
        finally:
            docx_path.unlink(missing_ok=True)
            temp_dir.rmdir()

    def convert_file(self, input_path):
        """根据扩展名选择转换方法"""
        input_path = Path(input_path)
        ext = input_path.suffix.lower()

        try:
            if ext == ".pdf":
                return self.convert_pdf(input_path)
            elif ext == ".docx":
                return self.convert_docx(input_path)
            elif ext in (".html", ".htm"):
                return self.convert_html(input_path)
            elif ext in (".xls", ".xlsx"):
                return self.convert_excel(input_path)
            elif ext == ".doc":
                if not os.path.exists(self.yozo_path["cmd"]):
                    raise RuntimeError("未找到永中Office安装路径")
                return self.convert_doc_with_yozo(input_path)
            else:
                raise ValueError(f"不支持的格式: {ext}")
        except Exception as e:
            raise RuntimeError(f"文件 {input_path.name} 转换失败: {str(e)}")

    def batch_convert(self, input_dir, output_dir, max_workers=4):
        """批量转换文档"""
        input_dir = Path(input_dir)
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)

        # 收集所有支持的文件
        file_list = []
        for ext in self.supported_ext:
            file_list.extend(list(input_dir.rglob(f"*{ext}")))

        if not file_list:
            logging.warning(f"未找到可转换文件于: {input_dir}")
            return

        logging.info(f"开始转换 {len(file_list)} 个文件...")

        def process_file(file_path):
            try:
                output_path = (
                    output_dir / f"{self.sanitize_filename(file_path.stem)}.md"
                )
                content = self.convert_file(file_path)
                output_path.write_text(content, encoding="utf-8")
                logging.info(f"转换成功: {file_path.name}")
                return True
            except Exception as e:
                logging.error(f"转换失败 {file_path.name}: {str(e)}")
                return False

        # 并行处理
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(
                tqdm(
                    executor.map(process_file, file_list),
                    total=len(file_list),
                    desc="转换进度",
                )
            )

        success_count = sum(results)
        logging.info(f"转换完成! 成功: {success_count}/{len(file_list)}")


INPUT_DIR = "./in"  # 替换为输入目录
OUTPUT_DIR = "./out"  # 替换为输出目录
converter = DocumentConverter()
converter.batch_convert(INPUT_DIR, OUTPUT_DIR)

2025-08-04 15:17:19,848 - INFO - 开始转换 17 个文件...
2025-08-04 15:17:20,349 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  附件2 中国人寿养老保险股份有限公司陕西省分公司划款申请单.pdf
2025-08-04 15:17:20,462 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  正文 关于印发《中国人寿养老保险股份有限公司陕西省分公司财务支出管理办法（2025年修订）》的通知.pdf
转换进度:   0%|          | 0/17 [00:00<?, ?it/s]2025-08-04 15:17:21,499 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  附件6 中国人寿养老保险股份有限公司陕西省分公司因公加班餐费审批单.pdf
2025-08-04 15:17:21,537 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  附件3 中国人寿养老保险股份有限公司陕西省分公司出差报告单.pdf
2025-08-04 15:17:22,202 - INFO - 转换成功: 国寿养老险沪发〔2025〕13号  附件1 中国人寿养老保险股份有限公司上海市分公司财务支出管理实施办法.pdf
转换进度:  18%|█▊        | 3/17 [00:01<00:07,  1.87it/s]2025-08-04 15:17:22,652 - ERROR - 转换失败 国寿养老险陕发〔2025〕52号  附件5 中国人寿养老保险股份有限公司陕西省分公司代理手续费签报表.docx: 文件 国寿养老险陕发〔2025〕52号  附件5 中国人寿养老保险股份有限公司陕西省分公司代理手续费签报表.docx 转换失败: DOCX转换失败: name 'CT_P' is not defined
2025-08-04 15:17:22,703 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  意见清单.html
2025-08-04 15:17:22,760 - INFO - 转换成功: 国寿养老险陕发〔2025〕52号  打印稿纸.html
2025-08-04 15:17:22,854 - INFO - 转换成功: 国

In [ ]:
import os
import subprocess
from pathlib import Path


def convert_with_yozo(input_path, output_path, target_format):
    """
    使用永中Office命令行转换文档
    :param input_path: 输入文件路径
    :param output_path: 输出文件路径
    :param target_format: 目标格式（支持：docx, pdf, html等）
    """
    # 永中Office命令行工具路径（根据实际安装位置调整）
    yozo_cmd = r"C:\Program Files (x86)\Yozosoft\Yozo Office\jre\bin\java.exe"
    jar_path = r"C:\Program Files (x86)\Yozosoft\Yozo Office\program\Yozo_Office.jar"

    cmd = [
        yozo_cmd,
        "-jar",
        jar_path,
        "-convert",
        str(input_path),
        str(output_path),
        "-format",
        target_format,
    ]

    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"永中转换失败: {e.stderr.decode('gbk')}")


def doc_to_markdown_with_yozo(doc_path):
    """通过永中Office将DOC转换为Markdown（两步转换：DOC->DOCX->MD）"""
    # 创建临时目录
    temp_dir = Path(tempfile.mkdtemp())
    docx_path = temp_dir / f"{Path(doc_path).stem}.docx"

    # 第一步：DOC -> DOCX
    convert_with_yozo(doc_path, docx_path, "docx")

    try:
        # 第二步：DOCX -> Markdown
        md_content = pypandoc.convert_file(str(docx_path), "md", format="docx")
        return md_content
    finally:
        # 清理临时文件
        docx_path.unlink(missing_ok=True)
        temp_dir.rmdir()


# 在批量转换函数中使用
def batch_convert_to_markdown(input_dir, output_dir):
    for file_path in Path(input_dir).rglob("*"):
        if file_path.suffix.lower() == ".doc":
            try:
                output_path = Path(output_dir) / f"{file_path.stem}.md"
                content = doc_to_markdown_with_yozo(file_path)
                output_path.write_text(content, encoding="utf-8")
                print(f"转换成功: {file_path.name}")
            except Exception as e:
                print(f"转换失败 {file_path.name}: {str(e)}")

In [ ]:
import os
import subprocess
import tempfile
import re
import html2text
import pandas as pd
import fitz  # PyMuPDF
from pathlib import Path
from bs4 import BeautifulSoup
from mammoth import convert_to_html


class Converter:
    def __init__(self, libreoffice_path=None):
        """
        初始化文档转换器
        :param libreoffice_path: LibreOffice可执行文件路径(默认为系统路径)
        """
        self.libreoffice_path = libreoffice_path or self._find_libreoffice()

    @staticmethod
    def _find_libreoffice():
        """尝试查找系统LibreOffice安装路径"""
        for path in [
            "/usr/bin/libreoffice",
            "/usr/bin/soffice",
            "/Applications/LibreOffice.app/Contents/MacOS/soffice",
            "C:\\Program Files\\LibreOffice\\program\\soffice.exe",
        ]:
            if os.path.exists(path):
                return path
        return "libreoffice"  # 使用系统PATH

    def convert_html_to_md(self, html_file, output_md=None):
        """
        将HTML转换为Markdown
        :param html_file: 输入的HTML文件路径
        :param output_md: 输出的MD文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        with open(html_file, "r", encoding="utf-8") as f:
            html_content = f.read()

        # 配置HTML转Markdown
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = True
        h.bypass_tables = False
        h.body_width = 0
        md_content = h.handle(html_content)

        # 清理多余的空行
        md_content = re.sub(r"\n{3,}", "\n\n", md_content)

        if output_md:
            with open(output_md, "w", encoding="utf-8") as f:
                f.write(md_content)
        return md_content

    def convert_xls_to_md(self, xls_file, output_md=None):
        """
        将Excel文件转换为Markdown表格
        :param xls_file: 输入的Excel文件路径(.xls, .xlsx)
        :param output_md: 输出的MD文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        # 读取Excel文件
        df = pd.read_excel(xls_file, sheet_name=0, header=None, dtype=str)

        # 处理空值
        df = df.fillna("")

        # 生成Markdown表格
        md_lines = []
        for i, row in enumerate(df.values):
            # 跳过全空行
            if all(cell == "" for cell in row):
                continue

            # 转换所有值为字符串
            row = [str(cell) for cell in row]

            # 表头分隔行
            if i == 0:
                md_lines.append("| " + " | ".join(row) + " |")
                md_lines.append("| " + " | ".join(["---"] * len(row)) + " |")
            else:
                md_lines.append("| " + " | ".join(row) + " |")

        md_content = "\n".join(md_lines)

        if output_md:
            with open(output_md, "w", encoding="utf-8") as f:
                f.write(md_content)
        return md_content

    def convert_pdf_to_md(self, pdf_file, output_md=None):
        """
        将PDF转换为Markdown
        :param pdf_file: 输入的PDF文件路径
        :param output_md: 输出的MD文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        md_content = ""
        with fitz.open(pdf_file) as doc:
            for page in doc:
                text = page.get_text("text")
                # 基本清理
                text = re.sub(r"\s+", " ", text)  # 合并多余空格
                text = re.sub(r"(\n\s*){3,}", "\n\n", text)  # 减少多余空行
                md_content += text.strip() + "\n\n"

        if output_md:
            with open(output_md, "w", encoding="utf-8") as f:
                f.write(md_content)
        return md_content

    def _convert_doc_to_docx(self, doc_file, output_dir=None):
        """
        使用LibreOffice将DOC转换为DOCX
        :param doc_file: 输入的DOC文件路径
        :param output_dir: 输出目录(默认临时目录)
        :return: 转换后的DOCX文件路径
        """
        if not output_dir:
            output_dir = tempfile.mkdtemp()

        cmd = [
            self.libreoffice_path,
            "--headless",
            "--convert-to",
            "docx",
            "--outdir",
            output_dir,
            doc_file,
        ]

        try:
            result = subprocess.run(
                cmd,
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=60,  # 设置超时时间
            )
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"LibreOffice转换失败: {e.stderr.decode()}") from e
        except subprocess.TimeoutExpired:
            raise RuntimeError("LibreOffice转换超时，请检查文件是否损坏") from None

        # 获取转换后的文件名
        base_name = Path(doc_file).stem
        docx_path = Path(output_dir) / f"{base_name}.docx"

        if not docx_path.exists():
            raise FileNotFoundError(f"转换后的文件未找到: {docx_path}")

        return str(docx_path)

    def _convert_complex_table(self, table_html):
        """
        将复杂的HTML表格转换为简化版的Markdown表格
        专注于使表格结构对大模型知识库更友好
        """
        soup = BeautifulSoup(table_html, "html.parser")
        table = soup.find("table")

        # 提取表格数据
        data = []
        for row in table.find_all("tr"):
            row_data = []
            for cell in row.find_all(["th", "td"]):
                # 处理跨行跨列
                rowspan = int(cell.get("rowspan", 1))
                colspan = int(cell.get("colspan", 1))

                # 获取单元格文本并清理
                cell_text = cell.get_text(separator=" ", strip=True)
                cell_text = re.sub(r"\s+", " ", cell_text)  # 合并多余空格

                # 对于跨列，重复内容填充
                for _ in range(colspan):
                    row_data.append(cell_text)

            if row_data:
                data.append(row_data)

        # 处理跨行 - 在后续行中添加占位符
        for i, row in enumerate(data):
            for j, cell in enumerate(row):
                # 如果单元格与上方相同，视为跨行内容
                if i > 0 and j < len(data[i - 1]) and cell == data[i - 1][j]:
                    # 使用特殊标记表示跨行内容
                    data[i][j] = "↑"  # 上箭头表示与上方单元格相同

        # 确保所有行长度一致
        max_cols = max(len(row) for row in data) if data else 0
        for row in data:
            while len(row) < max_cols:
                row.append("")

        # 生成Markdown表格
        if not data:
            return ""

        # 表头
        header = data[0]
        md_table = "| " + " | ".join(header) + " |\n"

        # 分隔线
        md_table += "| " + " | ".join(["---"] * len(header)) + " |\n"

        # 表格内容
        for row in data[1:]:
            md_table += "| " + " | ".join(row) + " |\n"

        return md_table

    def _convert_docx_html_to_md(self, html_content):
        """
        将DOCX转换的HTML转换为Markdown，特别处理复杂表格
        """
        # 使用BeautifulSoup解析HTML
        soup = BeautifulSoup(html_content, "html.parser")

        # 特殊处理表格
        for table in soup.find_all("table"):
            table_html = str(table)
            md_table = self._convert_complex_table(table_html)

            # 创建包含Markdown表格的pre标签
            table.replace_with(
                BeautifulSoup(f"<pre>\n{md_table}\n</pre>", "html.parser")
            )

        # 移除不需要的标签
        for elem in soup(["head", "script", "style", "meta", "link", "img"]):
            elem.decompose()

        # 转换清理后的HTML为Markdown
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = True  # 忽略图片
        h.bypass_tables = True  # 我们已经处理了表格
        h.body_width = 0  # 禁用自动换行

        # 获取清理后的HTML内容
        clean_html = str(soup)
        md_content = h.handle(clean_html)

        # 后处理
        md_content = re.sub(r"\n{3,}", "\n\n", md_content)  # 清理多余换行
        md_content = re.sub(r"(\|\s*↑\s*\|)", r"| ↑ |", md_content)  # 清理跨行标记空格

        return md_content

    def convert_docx_to_md(self, docx_file, output_md=None):
        """
        将DOCX转换为Markdown，优化表格处理
        :param docx_file: 输入的DOCX文件路径
        :param output_md: 输出的MD文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        with open(docx_file, "rb") as f:
            result = convert_to_html(f)

        html_content = result.value  # 获取转换后的HTML
        md_content = self._convert_docx_html_to_md(html_content)

        if output_md:
            with open(output_md, "w", encoding="utf-8") as f:
                f.write(md_content)
        return md_content

    def convert_doc_to_md(self, doc_file, output_md=None):
        """
        将DOC转换为Markdown (通过DOCX中间格式)
        :param doc_file: 输入的DOC文件路径
        :param output_md: 输出的MD文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        # 先转DOCX
        docx_file = self._convert_doc_to_docx(doc_file)
        # 再转Markdown
        return self.convert_docx_to_md(docx_file, output_md)

    def auto_convert(self, input_file, output_md=None):
        """
        自动根据文件扩展名选择转换方法
        :param input_file: 输入文件路径
        :param output_md: 输出文件路径(默认同目录)
        :return: 转换后的Markdown内容
        """
        ext = Path(input_file).suffix.lower()

        if not output_md:
            output_md = str(Path(input_file).with_suffix(".md"))

        converters = {
            ".html": self.convert_html_to_md,
            ".htm": self.convert_html_to_md,
            ".xls": self.convert_xls_to_md,
            ".xlsx": self.convert_xls_to_md,
            ".pdf": self.convert_pdf_to_md,
            ".docx": self.convert_docx_to_md,
            ".doc": self.convert_doc_to_md,
        }

        if ext not in converters:
            raise ValueError(f"不支持的文件类型: {ext}")

        return converters[ext](input_file, output_md)

In [11]:
input_dir = "./in"
output_dir = "./out"
conv = Converter()
for file_path in Path(input_dir).rglob("*"):
    conv.auto_convert(file_path)
# 示例转换
# conv._convert_doc_to_docx("./doc/test.doc", "./doc/output.docx")

# 单独转换示例
# conv.convert_html_to_md("input.html", "output.md")
# conv.convert_xls_to_md("data.xlsx", "table.md")
# conv.convert_pdf_to_md("report.pdf", "text.md")
# conv.convert_doc_to_md("old_document.doc", "converted.md")

In [10]:
input_dir = "./in"
print([i for i in Path(input_dir).rglob("*")])

[WindowsPath('in/云_18_费用支出标准.xls'), WindowsPath('in/沪_13_业务招待费支出申请单.doc'), WindowsPath('in/沪_13_加班用餐审批表.xls'), WindowsPath('in/沪_13_财务支出审批及报销流程简表.xls'), WindowsPath('in/沪_13_财务支出管理实施办法.pdf'), WindowsPath('in/沪_13_费用支出标准.xls'), WindowsPath('in/津_15_财务支出管理实施细则.doc'), WindowsPath('in/浙_29_费用支出标准.doc'), WindowsPath('in/陕_52_代理手续费签报表.docx'), WindowsPath('in/陕_52_出差报告单.pdf'), WindowsPath('in/陕_52_划款申请单.pdf'), WindowsPath('in/陕_52_因公加班餐费审批单.pdf'), WindowsPath('in/陕_52_意见清单.html'), WindowsPath('in/陕_52_打印稿纸.html'), WindowsPath('in/陕_52_财务支出管理办法.pdf'), WindowsPath('in/陕_52_费用支出标准.xls'), WindowsPath('in/陕_52_通知.pdf')]


In [9]:
input_dir = "./in"
conv = Converter()
for file_path in Path(input_dir).rglob("*.doc"):
    conv._convert_doc_to_docx(file_path, ".\in")

<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
C:\Users\zhangjingang\AppData\Local\Temp\ipykernel_3544\368385.py:4: SyntaxWarning: invalid escape sequence '\i'
  conv._convert_doc_to_docx(file_path, ".\in")


In [ ]:
import mammoth

result = mammoth.convert_to_html("./in/津_15_财务支出管理实施细则.docx")
with open("./out/津_15_财务支出管理实施细则.html", "w") as f:
    f.write(result.value)

In [ ]:
conv.convert_pdf_to_md(
    "./in/陕_52_财务支出管理办法.pdf", "./out/陕_52_财务支出管理办法.md"
)

'1 中国人寿养老保险股份有限公司陕西省 分公司财务支出管理办法 第一章 总 则 第一条 为加强陕西省分公司（以下简称“分公司”）财 务支出管理，提高费用管理的精细化、规范化水平，严格控 制成本支出，支持分公司运营，防范财务风险，根据《中华 人民共和国会计法》《企业会计准则》《企业会计准则—应用 指南》《金融企业财务规则》和《金融企业财务规则—实施 指南》等相关法律法规及制度的规定，结合分公司实际情况， 制定本办法。 第二条 本办法所称财务支出，是指分公司在经营过程 中发生的与业务、经营管理及其他相关的所有支出。 第三条 财务支出管理遵循的基本原则 （一）依法合规。深入落实中央八项规定精神，严格执 行各项财经法规和财务制度规定，各项财务支出必须合法合 规，从源头上有效防控财务风险。 （二）节俭增效。在保障分公司正常运营的基础上，强 化成本意识，增强财务约束，厉行节约，提升费用配置效率。 （三）预算管控。各项财务支出纳入预算管理，各部门 应采取有效举措，确保预算科学性和严肃性，提高预算执行 力，严格控制预算外支出。 （四）标准控制。按照实际需要和市场水平，合理确定\n\n2 并适时调整费用标准。标准一经制定，即须严格执行。本办 法各项费用标准所涉及的金额均为含税金额。 （五）归口管理。根据各部门职责权限，有关财务支出 由其归口管理部门负责，以便降低成本、明确责任主体、便 于管理。 （六）及时报销。各项财务支出应在经济事项发生后及 时办理报销手续，原则上不超过一个月，如超过一个月，须 提交书面说明并经部门主要负责人签字审批。 第四条 本办法适用于陕西省分公司。 第二章 职责分工 第五条 财务会计部履行费用综合管理职能，负责财务审 核，报销和会计核算，做好会计监督。 第六条 归口管理部门是归口费用的主管部门和责任部 门，履行归口费用统筹管理职能。根据工作职责，各部门归 口管理相应的公共费用支出。归口管理的公共费用支出由各 归口部门根据年度工作计划在年度预算内安排使用。 办公通讯费、水电费、财产保险费、学（协）会费、修 理费、车船使用费、绿化费、公杂费、租赁费、物业服务费、 职工工资、职工福利费、职工教育经费、社会保险、住房公 积金、劳动保护费、劳务派遣费、线路租用费、硬件维护费 （电子耗材费）、审计费、会议费、业务宣传费、印刷费、 邮寄费、中介机构和专业服务费、外事费、监

In [ ]:
import pdfplumber
import pandas as pd
import subprocess


def pdf_to_html(pdf_path, html_path):
    html = ["<html><body>"]

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取表格（优先）
            tables = page.extract_tables()
            for table in tables:
                html.append('<table class="pdf-table">')
                for row in table:
                    html.append("<tr>")
                    for cell in row:
                        html.append(f"<td>{cell}</td>")
                    html.append("</tr>")
                html.append("</table>")

            # 提取文本（保留结构）
            text = page.extract_text(x_tolerance=2, y_tolerance=2)
            for line in text.split("\n"):
                if line.upper() == line and len(line) < 50:  # 标题识别
                    html.append(f"<h2>{line}</h2>")
                else:
                    html.append(f"<p>{line}</p>")

    html.append("</body></html>")
    with open("temp.html", "w", encoding="utf-8") as f:
        f.write("\n".join(html))

    # 转换为 Markdown 并优化表格
    subprocess.run(["pandoc", "temp.html", "-t", "gfm", "-o", html_path])
    with open(html_path, "r+", encoding="utf-8") as f:
        content = f.read()
        content = re.sub(r"\|.*\|", fix_table_borders, content)  # 表格边框修正
        f.seek(0)
        f.write(content)


def fix_table_borders(match):
    table = match.group(0)
    if "---" not in table:  # 添加 Markdown 表格边框
        rows = table.split("\n")
        rows.insert(1, "|" + " --- |" * (rows[0].count("|") - 1))
        return "\n".join(rows)
    return table


pdf_to_html(
    ".\in\沪_13_财务支出管理实施办法.pdf", "./out/沪_13_财务支出管理实施办法.html"
)

<>:49: SyntaxWarning: invalid escape sequence '\i'
<>:49: SyntaxWarning: invalid escape sequence '\i'
C:\Users\zhangjingang\AppData\Local\Temp\ipykernel_3544\4027821327.py:49: SyntaxWarning: invalid escape sequence '\i'
  pdf_to_html(".\in\沪_13_财务支出管理实施办法.pdf", "./out/沪_13_财务支出管理实施办法.html")


In [ ]:
import win32com.client
import os
import time


def pdf_to_html(pdf_path, output_folder):
    """
    使用 Acrobat Pro 将 PDF 导出为 HTML
    :param pdf_path: PDF 文件完整路径
    :param output_folder: HTML 输出文件夹路径
    """
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    try:
        # 连接 Acrobat Pro
        acrobat = win32com.client.Dispatch("AcroExch.App")

        # 打开 PDF 文档
        pd_doc = win32com.client.Dispatch("AcroExch.PDDoc")
        if not pd_doc.Open(pdf_path):
            raise Exception(f"无法打开PDF文件: {pdf_path}")

        # 设置导出参数 (关键步骤)
        js_obj = pd_doc.GetJSObject()

        # 定义导出选项 (Acrobat JavaScript API)
        export_options = {
            "format": "com.adobe.acrobat.html",  # HTML格式
            "out": output_folder,  # 输出目录
            "createBookmarks": True,  # 创建书签
            "extractImages": True,  # 提取图片
            "includeAnnotations": True,  # 包含注释
        }

        # 执行导出操作
        js_obj.SaveAs(os.path.join(output_folder, "output.html"), export_options)

        print(f"导出成功! HTML 保存至: {output_folder}")

    except Exception as e:
        print(f"导出失败: {str(e)}")
    finally:
        # 清理资源
        pd_doc.Close()
        acrobat.Exit()
        time.sleep(1)  # 等待进程释放


if __name__ == "__main__":
    # 使用示例
    pdf_path = Path("./in/陕_52_财务支出管理办法.pdf")
    output_folder = r"./out"

    pdf_to_html(pdf_path, output_folder)

导出失败: 无法打开PDF文件: in\陕_52_财务支出管理办法.pdf


In [27]:
in_path = "./html/input.pdf"

## PDF Miner

In [ ]:
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams

with open("./html/pdfminer.html", "wb") as html_file:
    with open("./html/input.pdf", "rb") as pdf_file:
        extract_text_to_fp(
            pdf_file, html_file, output_type="html", laparams=LAParams(), codec="utf-8"
        )

## PyMuPDF

In [26]:
import fitz

doc = fitz.open("./html/input.pdf")
html = ""
for page in doc:
    html += page.get_text("html")  # 提取为HTML片段
with open("./html/pymupdf.html", "w", encoding="utf-8") as f:
    f.write(html)

In [ ]:
import pymupdf

doc = pymupdf.open("./html/input2.pdf")
pd_list = []
for page in doc:
    tables = page.find_tables()
    if tables.tables:
        for table in tables:
            pd_list.append(table.to_pandas())

pd_list[0]

,申请部门,Col1,单据日期,Col3
0,申请人,,实际出行人,
1,出发日期,,返回日期,
2,出差类型,□出差 □会议培训 □探亲,None,None
3,交通工具,□飞机 □火车 □汽车,None,None
4,出差事由,,None,None
5,备注,,None,None
6,部门负责人,,分公司主要负责人,


In [ ]:
import pymupdf

doc = pymupdf.open("./html/input3.pdf")
pd_list = []
for page in doc:
    tables = page.find_tables()
    if tables.tables:
        for table in tables:
            pd_list.append(table.to_pandas())

pd_list[0]

,申请部门：,： 年,月 日,附件 份
0,付款单位,,收款单位,
1,付款账号,,收款账号,
2,付款银行,,收款银行,
3,划款事项,□ 个人所得税/企业所得税 □人员薪酬费用 □增值税 □城建税等三小税 □水利基金 □其他,None,None
4,金额合计\n（大写）,（大写） 拾 万 仟 佰 拾 元 角 分,None,None
5,金额合计\n（小写）,￥：,None,None
6,备注,,None,None


## PDF Plumber

In [ ]:
import pdfplumber
import pandas as pd
import subprocess


def pdf_to_html(pdf_path, html_path):
    html = ["<html><body>"]

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取表格（优先）
            tables = page.extract_tables()
            for table in tables:
                html.append('<table class="pdf-table">')
                for row in table:
                    html.append("<tr>")
                    for cell in row:
                        html.append(f"<td>{cell}</td>")
                    html.append("</tr>")
                html.append("</table>")

            # 提取文本（保留结构）
            text = page.extract_text(x_tolerance=2, y_tolerance=2)
            for line in text.split("\n"):
                # if line.upper() == line and len(line) < 50:  # 标题识别
                #     html.append(f'<h2>{line}</h2>')
                # else:
                #     html.append(f'<p>{line}</p>')
                html.append(f"<p>{line}</p>")

    html.append("</body></html>")
    with open("temp.html", "w", encoding="utf-8") as f:
        f.write("\n".join(html))

    # 转换为 Markdown 并优化表格
    subprocess.run(["pandoc", "temp.html", "-t", "gfm", "-o", html_path])
    with open(html_path, "r+", encoding="utf-8") as f:
        content = f.read()
        # content = re.sub(r'\|.*\|', fix_table_borders, content)  # 表格边框修正
        f.seek(0)
        f.write(content)


# def fix_table_borders(match):
#     table = match.group(0)
#     if "---" not in table:  # 添加 Markdown 表格边框
#         rows = table.split('\n')
#         rows.insert(1, '|' + ' --- |' * (rows[0].count('|') - 1))
#         return '\n'.join(rows)
#     return table

In [ ]:
pdf_to_html(
    ".\in\沪_13_财务支出管理实施办法.pdf", "./out/沪_13_财务支出管理实施办法.html"
)

In [56]:
pdf_to_html("./html/input2.pdf", "./html/output2.md")
pdf_to_html("./html/input3.pdf", "./html/output3.md")

In [ ]:
import pandas as pd

pdf_path = "./html/input.pdf"
df_list = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # 提取表格（优先）
        tables = page.extract_tables()
        if tables:
            df_list.append(pd.DataFrame(tables[0]))
df_list[0]

,0,1
0,单笔支出金额,审批流程
1,2000 元（含）以下,经办人→经办部门负责人→会计审\n核→归口管理部门负责人→财务会\n计部负责人→分管财务的公司领导
2,2000 元以上,经办人→经办部门负责人→会计审\n核→归口管理部门负责人→财务会\n计部负责人→分管财务的公...


In [ ]:
tab_json = df_list[0]
tab_json.to_dict()

{0: {0: '单笔支出金额', 1: '2000 元（含）以下', 2: '2000 元以上'},
 1: {0: '审批流程',
  1: '经办人→经办部门负责人→会计审\n核→归口管理部门负责人→财务会\n计部负责人→分管财务的公司领导',
  2: '经办人→经办部门负责人→会计审\n核→归口管理部门负责人→财务会\n计部负责人→分管财务的公司领导\n→分公司主要负责人'}}

In [ ]:
import pandas as pd

pdf_path = "./html/input3.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # 提取表格（优先）
        tables = page.extract_tables()
        df = pd.DataFrame(tables[0])
df

,0,1,2,3
0,付款单位,,收款单位,
1,付款账号,,收款账号,
2,付款银行,,收款银行,
3,划款事项,□ 个人所得税/企业所得税 □人员薪酬费用 □增值税 □城建税等三小税 □水利基金 □其他,None,None
4,金额合计\n（大写）,（大写） 拾 万 仟 佰 拾 元 角 分,None,None
5,金额合计\n（小写）,￥：,None,None
6,备注,,None,None


In [39]:
import pandas as pd
import pdfplumber

pdf_path = "./html/input3.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # 提取表格（优先）
        tables = page.extract_tables()
        df = pd.DataFrame(tables[0])
        text = page.extract_text(x_tolerance=2, y_tolerance=2)
df

,0,1,2,3
0,付款单位,,收款单位,
1,付款账号,,收款账号,
2,付款银行,,收款银行,
3,划款事项,□ 个人所得税/企业所得税 □人员薪酬费用 □增值税 □城建税等三小税 □水利基金 □其他,None,None
4,金额合计\n（大写）,（大写） 拾 万 仟 佰 拾 元 角 分,None,None
5,金额合计\n（小写）,￥：,None,None
6,备注,,None,None


In [182]:
import re
def md_formater(str_in):
    if "章" in str_in:
        str_in = re.sub(
            r"(第[一二三四五六七八九十百]+章)",  # 只匹配数字/中文数字
            r"\n\n## \1 ",
            str_in,
        )
    if "条" in str_in:
        str_in = re.sub(
            r'(第[一二三四五六七八九十百]+条)',
            r"\n\n#### \1\n",
            str_in,
        )
    if "节" in str_in:
        str_in = re.sub(
            r'(第[一二三四五六七八九十百]+节)',
            r"\n\n### \1 ",
            str_in,
        )
    if "（" in str_in:
        str_in = re.sub(
            r'(（[一二三四五六七八九十百]+）)',
            r"\n\n\1",
            str_in,
        )
    if "." in str_in:
        str_in = re.sub(
            r'(\d+)\.',
            r"\n\n（\1）",
            str_in,
        )
    if "附件" in str_in:
        str_in = re.sub(
            r"(附件：)",
            r"\n\n## \1\n",
            str_in,
        )
    return str_in

In [12]:
md_formater('附件：\n1. 中国人寿养老保险股份有限公司陕西省分公')

'附件：\n\n1.  中国人寿养老保险股份有限公司陕西省分公'

In [43]:
def format_table(table):
    df = pd.DataFrame(table)
    df = df.fillna("")
    df = df.map(lambda str_in: "".join(str_in.split("\n")))
    md = df.to_markdown(index=False)
    md = re.sub(
        r"( {2,})",
        r" ",
        md
    )
    md = re.sub(
        r"(-{3,})",
        r"-----",
        md
    )
    intent = " "*0
    md = intent + f"\n{intent}".join(md.split("\n"))
    return f"\n{intent}\n" + md + f"\n{intent}\n"

In [42]:
def replace_table_in_text(tables, text_list):
    inserted_list = text_list
    for table in tables:
        table_inserted = format_table(table)
        head = table[0][0]
        tail = table[-1][-1].split("\n")[-1]
        h_idx = text_list.index(head)
        t_idx = len(text_list) - 1 - text_list[::-1].index(tail)
        inserted_list = text_list[:h_idx] + [table_inserted] + text_list[t_idx + 1:]
    return inserted_list

In [ ]:
import pandas as pd
import pdfplumber

pdf_path = "./html/input3.pdf"

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    tables = page.extract_tables()

    # words = page.extract_words(x_tolerance=3, y_tolerance=3)
    # text_list = [w["text"] for w in words]
    # replaced_list = replace_table_in_text(tables, "aaaa", text_list)
    # print(replaced_list)
    tab = pd.DataFrame(tables[0])
tab = tab.fillna("")
tab = tab.map(lambda str_in: " ".join(str_in.split("\n")))

tab.to_markdown(index=False)
# tab

| 0                 | 1                                                                             | 2        | 3   |
|:------------------|:------------------------------------------------------------------------------|:---------|:----|
| 付款单位          |                                                                               | 收款单位 |     |
| 付款账号          |                                                                               | 收款账号 |     |
| 付款银行          |                                                                               | 收款银行 |     |
| 划款事项          | □ 个人所得税/企业所得税 □人员薪酬费用 □增值税 □城建税等三小税 □水利基金 □其他 |          |     |
| 金额合计 （大写） | （大写） 拾 万 仟 佰 拾 元 角 分                                              |          |     |
| 金额合计 （小写） | ￥：                                                                          |          |     |
| 备注              |                                                                               |          |     |


In [62]:
" ".join(tab.iat[4,0].split("\n"))

'金额合计 （大写）'

In [183]:
import pandas as pd
import pdfplumber

pdf_path = "./html/input.pdf"

with pdfplumber.open(pdf_path) as pdf:
    page_list = []
    for page in pdf.pages:
        num = page.page_number
        word = page.extract_words(x_tolerance=3, y_tolerance=3)
        tables = page.extract_tables()
        text_list = [w.get("text") for w in word]
        text_list = replace_table_in_text(tables, text_list)
        text_list.pop()
        text_list = [md_formater(l) for l in text_list]
        if num == 1:
            text_list[0] = "# " + text_list[0]
        # print(text_list)
        page_list.append("".join(text_list))
    print("".join(page_list))
    content = "".join(page_list)
with open("./html/output.md", "w", encoding="utf-8") as f:
    f.write(content)

# 中国人寿养老保险股份有限公司陕西省分公司财务支出管理办法

## 第一章 总则

#### 第一条
为加强陕西省分公司（以下简称“分公司”）财务支出管理，提高费用管理的精细化、规范化水平，严格控制成本支出，支持分公司运营，防范财务风险，根据《中华人民共和国会计法》《企业会计准则》《企业会计准则—应用指南》《金融企业财务规则》和《金融企业财务规则—实施指南》等相关法律法规及制度的规定，结合分公司实际情况，制定本办法。

#### 第二条
本办法所称财务支出，是指分公司在经营过程中发生的与业务、经营管理及其他相关的所有支出。

#### 第三条
财务支出管理遵循的基本原则

（一）依法合规。深入落实中央八项规定精神，严格执行各项财经法规和财务制度规定，各项财务支出必须合法合规，从源头上有效防控财务风险。

（二）节俭增效。在保障分公司正常运营的基础上，强化成本意识，增强财务约束，厉行节约，提升费用配置效率。

（三）预算管控。各项财务支出纳入预算管理，各部门应采取有效举措，确保预算科学性和严肃性，提高预算执行力，严格控制预算外支出。

（四）标准控制。按照实际需要和市场水平，合理确定并适时调整费用标准。标准一经制定，即须严格执行。本办法各项费用标准所涉及的金额均为含税金额。

（五）归口管理。根据各部门职责权限，有关财务支出由其归口管理部门负责，以便降低成本、明确责任主体、便于管理。

（六）及时报销。各项财务支出应在经济事项发生后及时办理报销手续，原则上不超过一个月，如超过一个月，须提交书面说明并经部门主要负责人签字审批。

#### 第四条
本办法适用于陕西省分公司。

## 第二章 职责分工

#### 第五条
财务会计部履行费用综合管理职能，负责财务审核，报销和会计核算，做好会计监督。

#### 第六条
归口管理部门是归口费用的主管部门和责任部门，履行归口费用统筹管理职能。根据工作职责，各部门归口管理相应的公共费用支出。归口管理的公共费用支出由各归口部门根据年度工作计划在年度预算内安排使用。办公通讯费、水电费、财产保险费、学（协）会费、修理费、车船使用费、绿化费、公杂费、租赁费、物业服务费、职工工资、职工福利费、职工教育经费、社会保险、住房公积金、劳动保护费、劳务派遣费、线路租用费、硬件维护费（电子耗材费）、审计费、会议费、业务宣传费、印刷费、邮寄费、中介

In [2]:
import pandas as pd
import pdfplumber

pdf_path = "./html/input.pdf"

with pdfplumber.open(pdf_path) as pdf:
    last = pdf.pages[-1]
    print([l["text"] for l in last.extract_words(x_tolerance=3, y_tolerance=3)])

['第十章', '税金及附加', '第九十七条', '税金及附加是指分公司管理的增值税及附', '加、企业所得税、房产税、印花税、车船税、城镇土地使用', '税、土地增值税、其他地方性税费等。', '第十一章', '附', '则', '第九十八条', '本办法由财务会计部负责解释和修订。', '第九十九条', '本办法自印发之日起施行。《中国人寿养老', '保险股份有限公司陕西省分公司财务支出管理办法（2022', '年', '修订）》（国寿养老险陕发〔2022〕38', '号）同时废止。', '附件：1.中国人寿养老保险股份有限公司陕西省分公', '司划款申请单', '2.中国人寿养老保险股份有限公司陕西省分公', '司出差报告单', '3.中国人寿养老保险股份有限公司陕西省分公', '司费用支出标准', '4.中国人寿养老保险股份有限公司陕西省分公', '司代理手续费签报表', '5.中国人寿养老保险股份有限公司陕西省分公', '司因公加班餐费审批单', '31']


In [ ]:
with open("./html/output4.html", "w", encoding="utf-8") as f:
    f.write(df.to_html(header=False, index=False))

In [14]:
df

,0,1,2,3,4
0,部 门,,None,日期,
1,涉及的工作\n内容,,None,None,None
2,人员（分公\n司员工，含\n交流、借调\n人员，人员\n较多可另附\n明细）,总人数： 人,None,None,None
3,None,姓 名,起始时间,None,餐 别
4,None,,,None,□早餐 □午餐 □晚餐
5,None,,,None,□早餐 □午餐 □晚餐
6,None,,,None,□早餐 □午餐 □晚餐
7,None,,,None,□早餐 □午餐 □晚餐
8,None,,,None,□早餐 □午餐 □晚餐
9,None,,,None,□早餐 □午餐 □晚餐


In [ ]:
# 转换为 Markdown 并优化表格
html_path = "./html/output4.md"
subprocess.run(["pandoc", "./html/output4.html", "-t", "gfm", "-o", html_path])
with open(html_path, "r+", encoding="utf-8") as f:
    content = f.read()
    # content = re.sub(r'\|.*\|', fix_table_borders, content)  # 表格边框修正
    f.seek(0)
    f.write(content)

In [ ]:
import pdfplumber

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # 提取表格（优先）
        tables = page.extract_tables()
        for table in tables:
            html.append('<table class="pdf-table">')
            for row in table:
                html.append("<tr>")
                for cell in row:
                    html.append(f"<td>{cell}</td>")
                html.append("</tr>")
            html.append("</table>")

        # 提取文本（保留结构）
        text = page.extract_text(x_tolerance=2, y_tolerance=2)
        for line in text.split("\n"):
            # if line.upper() == line and len(line) < 50:  # 标题识别
            #     html.append(f'<h2>{line}</h2>')
            # else:
            #     html.append(f'<p>{line}</p>')
            html.append(f"<p>{line}</p>")

# XLS

doc
['output.docx']
['converted.md', 'output.md', 'test.doc']
doc\output.docx
[]
['test.docx']


In [9]:
import subprocess
from pathlib import Path
import os
def xls2pdf(input_path, output_path):
    """通过命令行直接转换"""
    cmd = [
        "C:\\Program Files\\LibreOffice\\program\\soffice.exe",
        '--headless',
        '--convert-to', 'pdf',
        '--outdir', os.path.dirname(output_path),
        input_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ 转换成功: {output_path}")
    else:
        print(f"❌ 转换失败: {result.stderr}")


In [10]:
xls2pdf("./xls/云_18_费用支出标准.xls", "./xls/云_18_费用支出标准.pdf")

✅ 转换成功: ./xls/云_18_费用支出标准.pdf


In [19]:
for in_path in Path("./xls").glob("*.xls"):
    in_path = in_path.resolve()
    out_path = in_path.with_suffix(".pdf")
    xls2pdf(in_path,out_path)

✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\云_18_费用支出标准.pdf
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_加班用餐审批表.pdf
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_财务支出审批及报销流程简表.pdf
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_费用支出标准.pdf
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\陕_52_费用支出标准.pdf


In [22]:
in_list = [in_path for in_path in Path("./xls").glob("*.xls")]
for idx, f in enumerate(in_list):
    print(f"[{idx}]  {f}")

[0]  xls\云_18_费用支出标准.xls
[1]  xls\沪_13_加班用餐审批表.xls
[2]  xls\沪_13_财务支出审批及报销流程简表.xls
[3]  xls\沪_13_费用支出标准.xls
[4]  xls\陕_52_费用支出标准.xls


In [97]:
import pandas as pd
import re
df = pd.read_excel(in_list[2],header=None)
df = df.dropna(how='all')
df = df.fillna(r"`merged/none`")
# df = df.replace("", r"`none`")
# df = df.map(lambda str_in: "".join(str_in.split("\n")))
md = df.to_markdown(index=False)
md = re.sub(
    r"( {2,})",
    r" ",
    md
)
md = re.sub(
    r"(-{3,})",
    r"-----",
    md
)
with open(in_list[2].with_suffix(".md"), "w", encoding="utf-8") as f_md:
    f_md.write(md)

In [34]:
df.iat[2,10]

'分公司主要负责人'

In [21]:
import re
def format_table_merged(df):
    df = df.fillna("")
    # df = df.replace("", r"`none`")
    df = df.map(lambda str_in: "".join(str_in.split("\n")))
    md = df.to_markdown(index=False)
    md = re.sub(
        r"( {2,})",
        r" ",
        md
    )
    md = re.sub(
        r"(-{3,})",
        r"-----",
        md
    )
    intent = " "*0
    md = intent + f"\n{intent}".join(md.split("\n"))
    return f"\n{intent}\n" + md + f"\n{intent}\n"

In [2]:
import pandas as pd
import pdfplumber
from pathlib import Path
pdf_path = Path("./xls/沪_13_财务支出审批及报销流程简表.pdf")

with pdfplumber.open(pdf_path) as pdf:
    page_list = []
    for page in pdf.pages:
        num = page.page_number
        # word = page.extract_words(x_tolerance=3, y_tolerance=3)
        tables = page.extract_tables(table_settings={ "join_tolerance": 20})
        df = pd.DataFrame(tables[0])
df
    #     print(tables[0])
    # md = format_table_merged(tables[0])
    # print(md)
    # with open(pdf_path.with_suffix(".md"), "w", encoding="utf-8") as f_md:
    #     f_md.write(md)
    #     text_list = [w.get("text") for w in word]
    #     tables = page.extract_tables()
    #     print(tables[0])
    #     if num == 1:
    #         text_list[0] = "# " + text_list[0]
    #     page_list.append("".join(text_list))
    # print("".join(page_list))
#     content = "".join(page_list)
# with open("./xls/沪_13_财务支出审批及报销流程简表2.md", "w", encoding="utf-8") as f:
#     f.write(content)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,类别,流程,支出类别,经办人员,经办部门\n负责人,归口管理\n部门主要\n负责人,财务审核\n工作人员,财务会计部\n主要负责人,分管经办部门的\n总经理室成员,分管财务的总经\n理室成员,分公司\n主要负责人,总公司核准审批,出纳检查,适用范围
1,直\n接\n报\n销\n类\n支\n出,无需事前以签报形\n式履行审批手续，\n由经办部门在经济\n事项发生后直接按\n照规定的审批...,预算内部门费用支出\n2万元以下,√,√,√,√,√,√,,,,√,业务招待费、公杂费（办公用品归口综合管理部）等
2,None,None,预算内部门费用支出\n2万元以上（含2万元）,√,√,√,√,√,√,√,√,,√,None
3,None,None,预算内归口费用支出\n2万元以下,√,√,√,√,√,√,,,,√,综合管理部：主要包括：办公通讯费、水电费、办公用\n品、修理费、车船使用费、绿化费、租赁费、...
4,None,None,预算内归口费用支出\n2万元以上（含2万元）,√,√,√,√,√,√,√,√,,√,None
5,,事前审批环节,审批权限,,单笔支出2万元以下,None,None,None,None,单笔支出2万元——15万元\n（含2万元）,None,,,
6,事\n前\n审\n批\n类,None,预算内费用支出,√,√,√,√,√,√,√,√,√,√,除直接报销类外的预算内费用支出，主要包括差旅费、\n会议费、外事费、业务宣传费、劳动保护费、...
7,支\n出,None,,审批权限,None,None,None,None,None,None,None,None,,
8,(\n以\n签\n报\n形\n式\n),None,预算外支出,预算外支出，不论金额大小，一律事前以签报形式上报，并经分公司党委会审议。,None,None,None,None,None,None,None,None,None,预算外支出
9,None,None,捐赠支出,捐赠支出，不论金额大小，均须事前上报总公司审批。,None,None,None,None,None,None,None,None,None,捐赠支出


In [7]:
import camelot

ctabs= camelot.io.read_pdf(pdf_path, pages='1',flavor='lattice',strip_text = '\n')####从1开始计数
ctab=ctabs[0]
# camelot.plot(ctab, kind='line').show()


In [22]:
cmd = ctab.df
print(format_table_merged(cmd))



| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |
|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| 类别 | 流程 | 支出类别 | 经办人员 | 经办部门负责人 | 归口管理部门主要负责人 | 财务审核工作人员 | 财务会计部主要负责人 | 分管经办部门的总经理室成员 | 分管财务的总经理室成员 | 分公司主要负责人 | 总公司核准审批 | 出纳检查 | 适用范围 |
| 直接报销类支出 | 无需事前以签报形式履行审批手续，由经办部门在经济事项发生后直接按照规定的审批流程办理报销手续的财务支出 | 预算内部门费用支出2万元以下 | √ | √ | √ | √ | √ | √ | | | | √ | 业务招待费、公杂费（办公用品归口综合管理部）等 |
| | | 预算内部门费用支出2万元以上（含2万元） | √ | √ | √ | √ | √ | √ | √ | √ | | √ | |
| | | 预算内归口费用支出2万元以下 | √ | √ | √ | √ | √ | √ | | | | √ | 综合管理部：主要包括：办公通讯费、水电费、办公用品、修理费、车船使用费、绿化费、租赁费、物业管理费、职工工资、职工福利费、社会保险、住房公积金、劳动保险费、代缴个人所得税、劳务派遣费、残疾人就业保障金、线路租用费、电子维护费，根据合同需定期结算的办公行政费用等。财务会计部：银行结算费、税金及附加，税金及附加是指分公司管理的增值税及附加、企业所得税、房产税、印花税、车船税、城镇土地使用税、土地增值税、其他地方性税费等 |
| | | 预算内归口费用支出2万元以上（含2万元） | √ | √ | √ | √ | √ | √ | √ | √ | | √ | |
| 事前审批类支出(以签报形式) | 事前审批环节 | 审批权限 | | 单笔支出2万元以下 | | | | | 单笔支出2万元——15万元 （含2万元） | | | | |
| | | 预算内费用支出 | √ | √ | √ | √ | √ | √ | √ | √ | √ | √ | 除直接

In [ ]:
from bs4 import BeautifulSoup
import re
from lxml import html

def get_first_table(soup):
    # 1. 只保留第一个table标签
    first_table = soup.find('table')    
    # 2. 创建只包含这个table的新文档
    new_soup = BeautifulSoup(features='xml')
    new_soup.append(first_table)
    return new_soup

def remove_trailing_empty_cells(table):
    """移除表格每行末尾共有的空白单元格"""
    trailing_empty_counts = []
    
    for tr in table.find_all('tr'):
        cells = tr.find_all(['td', 'th'])
        empty_cnt = 0
        
        # 从后向前计算空白单元格数量
        for cell in reversed(cells):
            if not cell.text.strip():
                empty_cnt += 1
            else:
                break
                
        trailing_empty_counts.append(empty_cnt)
    
    if not trailing_empty_counts:
        return 0
    
    common_empty_cnt = min(trailing_empty_counts)
    
    if common_empty_cnt > 0:
        for tr in table.find_all('tr'):
            cells = tr.find_all(['td', 'th'])
            for _ in range(common_empty_cnt):
                if cells:
                    cells[-1].decompose()
                    cells = cells[:-1]
    
    return common_empty_cnt

def flatten_paragraphs(soup):
    """去除p标签，只保留文本内容在td中"""
    for p in soup.find_all('p'):
        # 获取p标签内的所有文本（包括子孙元素的文本）
        text_content = p.get_text(' ', strip=True)
        
        # 用纯文本替换整个p标签
        p.replace_with(text_content)

def remove_empty_rows(soup):
    """移除表格中的空行并简化空白单元格"""
    for tr in soup.find_all('tr'):
        # 先处理空白单元格
        for cell in tr.find_all(['td', 'th']):
            if not cell.text.strip():
                # 创建新的自闭合标签
                new_cell = soup.new_tag(cell.name)
                # 保留原有属性
                new_cell.attrs = cell.attrs.copy()
                cell.replace_with(new_cell)
        
        # 然后检查是否整行为空
        if not tr.text.strip() or all(cell.name in ['td', 'th'] and not cell.contents 
           for cell in tr.find_all(['td', 'th'])):
            tr.decompose()

def add_basic_table_styles(soup):
    """为表格添加基础样式"""
    for table in soup.find_all('table'):
        table['border'] = '1'
        table['cellspacing'] = '0'
        table['cellpadding'] = '4'

def remove_unnecessary_elements(soup):
    """移除不必要的元素"""
    elements_to_remove = ['style', 'script', 'comment', 'head', 'meta', 'link', 'colgroup', 'col', 'title']
    for element in soup.find_all(elements_to_remove):
        element.decompose()

def remove_style_attributes(soup):
    """移除样式和类属性"""
    for tag in soup.find_all(True):
        for attr in tag.attrs:
            if not attr in ['colspan', 'rowspan']:
                del tag[attr]

def prettify(soup, compact = False):
    out = soup.prettify()
    tab = "  "

    out = re.sub(
        r"(</?t[rdable].*?>)\n\s+",
        r"\1\n",
        out
    )
    out = re.sub(
        r"\s+(</t[rdable]>)",
        r"\n\1",
        out
    )
    out = re.sub(
        r"\n   ",
        r"\n",
        out
    )
    out = re.sub(
        r"(</?t[rdable].*?>)\n\s+",
        r"\1\n",
        out
    )
    out = re.sub(
        r"([^>])\n([^<])",
        r"\1\2",
        out
    )
    out = re.sub(
        r"(<td.*?>)\n(.*?)\n(</td>)",
        r"\1\2\3",
        out
    )
    out = re.sub(
        r"<td",
        tab*2 + r"<td",
        out
    )
    out = re.sub(
        r"(</*tr)",
        tab + r"\1",
        out
    )

    if compact:
        out = re.sub(
            r"(</*)t([drh].*?>)",
            r"\1\2",
            out
        )
    return out
            

def medium_simplify(soup):
    for tag in soup.find_all(True):
        attrs = {}
        for attr in ['colspan', 'rowspan']:
            if attr in tag.attrs:
                attrs[attr] = tag[attr]
        tag.attrs = attrs
    output = prettify(soup)
    output = re.sub(r"<\?xml.*?\?>", "", output).strip()
    return output

def ultra_simplify(soup):
    """极简表格压缩方案"""
    # 1. 只保留colspan/rowspan属性
    for tag in soup.find_all(True):
        attrs = {}
        for attr in ['colspan', 'rowspan']:
            if attr in tag.attrs:
                attrs[attr] = tag[attr]
        tag.attrs = attrs
    
    output = prettify(soup, compact = True)
    # output = re.sub(r'>\s*<', '><', output)  # 移除标签间空白
    # output = re.sub(r'\s+', ' ', output)     # 压缩连续空白
    output = re.sub(r"<\?xml.*?\?>", "", output)
    output = r"<!-- 标签映射表={<table>:<t>,<tr>:<r>,<td>:<d>,<th>:<h>} -->" + output
    return output

def simplify_xhtml_table(xhtml_content, in_browser = True, compact = False):
    """主函数：简化XHTML表格内容"""
    soup = BeautifulSoup(xhtml_content, 'lxml-xml')

    # 处理
    remove_unnecessary_elements(soup)
    remove_style_attributes(soup)
    flatten_paragraphs(soup)
    remove_empty_rows(soup)
    
    for table in soup.find_all('table'):
        remove_trailing_empty_cells(table)
    
    add_basic_table_styles(soup)

    if in_browser:
        return re.sub(r'\s+', ' ', str(soup))
    elif not compact:
        output = medium_simplify(get_first_table(soup))
    else:
        output = ultra_simplify(get_first_table(soup))
    return output

if __name__ == '__main__':
    with open('./xls/沪_13_财务支出审批及报销流程简表.html', 'r', encoding='utf-8') as f:
        xhtml_content = f.read()

    simplified_xhtml = simplify_xhtml_table(xhtml_content, in_browser=False, compact=True)

    with open('./xls/output2.html', 'w', encoding='utf-8') as f:
        f.write(simplified_xhtml)

FileNotFoundError: [Errno 2] No such file or directory: './xls/沪_13_财务支出审批及报销流程简表.xhtml'

笔记：
评测-》应用了什么方法，在哪些情况有提升
ab test（后验），挑几十个问题去问（>30），评分-》客观对比，先做一个

感受和体会：对于哪些情况很有效-》对自己有用，评测展示给其他人，
记录灵感来源-》自己全套讲
汇报：讲出价值和意义，有什么价值，提升，跨场景+经验迁移，中等场景验证方法优越性-》扩展

为什么做这个事情，有什么价值，方法本身+经验迁移的价值，

In [13]:
import subprocess
from pathlib import Path
import os
def xls2xhtml(input_path):
    """通过命令行直接转换"""
    cmd = [
        "C:\\Program Files\\LibreOffice\\program\\soffice.exe",
        '--headless',
        '--convert-to', 'html',
        '--outdir', os.path.dirname(input_path),
        input_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ 转换成功: {input_path}")
    else:
        print(f"❌ 转换失败: {result.stderr}")
        
for in_path in Path("./xls").glob("*.xls"):
    in_path = in_path.resolve()
    xls2xhtml(in_path)



✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\云_18_费用支出标准.xls
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_加班用餐审批表.xls
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_财务支出审批及报销流程简表.xls
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\沪_13_费用支出标准.xls
✅ 转换成功: C:\Users\zhangjingang\Desktop\临时工作材料_实习生温超群\开发\project\convert\xls\陕_52_费用支出标准.xls


In [ ]:
from bs4 import BeautifulSoup
import re
from lxml import html

def get_first_table(soup):
    # 1. 只保留第一个table标签
    first_table = soup.find('table')    
    # 2. 创建只包含这个table的新文档
    new_soup = BeautifulSoup(features='xml')
    new_soup.append(first_table)
    remove_style_attributes(new_soup)
    return new_soup

def remove_trailing_empty_cells(table):
    """移除表格每行末尾共有的空白单元格"""
    trailing_empty_counts = []
    
    for tr in table.find_all('tr'):
        cells = tr.find_all(['td', 'th'])
        empty_cnt = 0
        
        # 从后向前计算空白单元格数量
        for cell in reversed(cells):
            if not cell.text.strip():
                empty_cnt += 1
            else:
                break
                
        trailing_empty_counts.append(empty_cnt)
    
    if not trailing_empty_counts:
        return 0
    
    common_empty_cnt = min(trailing_empty_counts)
    
    if common_empty_cnt > 0:
        for tr in table.find_all('tr'):
            cells = tr.find_all(['td', 'th'])
            for _ in range(common_empty_cnt):
                if cells:
                    cells[-1].decompose()
                    cells = cells[:-1]
    
    return common_empty_cnt

def flatten_paragraphs(soup, label):
    """去除p标签，只保留文本内容在td中"""
    for l in soup.find_all(label):
        # 获取p标签内的所有文本（包括子孙元素的文本）
        text_content = l.get_text(' ', strip=True)
        
        # 用纯文本替换整个p标签
        l.replace_with(text_content)

def remove_empty_rows(soup):
    """移除表格中的空行并简化空白单元格"""
    for tr in soup.find_all('tr'):
        # 先处理空白单元格
        for cell in tr.find_all(['td', 'th']):
            if not cell.text.strip():
                # 创建新的自闭合标签
                new_cell = soup.new_tag(cell.name)
                # 保留原有属性
                new_cell.attrs = cell.attrs.copy()
                cell.replace_with(new_cell)
        
        # 然后检查是否整行为空
        if not tr.text.strip() or all(cell.name in ['td', 'th'] and not cell.contents 
           for cell in tr.find_all(['td', 'th'])):
            tr.decompose()

def add_basic_table_styles(soup):
    """为表格添加基础样式"""
    for table in soup.find_all('table'):
        table['border'] = '1'
        table['cellspacing'] = '0'
        table['cellpadding'] = '4'

def remove_unnecessary_elements(soup):
    """移除不必要的元素"""
    elements_to_remove = ['style', 'script', 'comment', 'head', 'meta', 'link', 'colgroup', 'col', 'title']
    for element in soup.find_all(elements_to_remove):
        element.decompose()

def remove_style_attributes(soup):
    """移除样式和类属性"""
    for tag in soup.find_all(True):
        attrs = {}
        for attr in ['colspan', 'rowspan']:
            if attr in tag.attrs:
                attrs[attr] = tag[attr]
        tag.attrs = attrs

def prettify(soup, compact = False):
    out = soup.prettify()
    tab = " "

    out = re.sub(
        r"(</?t[rdable].*?>)\n\s+",
        r"\1\n",
        out
    )
    out = re.sub(
        r"\s+(</t[rdable]>)",
        r"\n\1",
        out
    )
    # out = re.sub(
    #     r"\n   ",
    #     r"\n",
    #     out
    # )
    # out = re.sub(
    #     r"(</?t[rdable].*?>)\n\s+",
    #     r"\1\n",
    #     out
    # )
    # out = re.sub(
    #     r"([^>])\n([^<])",
    #     r"\1\2",
    #     out
    # )
    out = re.sub(
        r"(<td.*?>)\n(.*?)\n(</td>)",
        r"\1\2\3",
        out
    )
    out = re.sub(
        r"(<td.*?>)\n(</td>)",
        r"\1\2",
        out
    )
    out = re.sub(
        r"<td",
        tab*2 + r"<td",
        out
    )
    out = re.sub(
        r"(</*tr)",
        tab + r"\1",
        out
    )

    if compact:
        out = re.sub(
            r"(</*)t([drh].*?>)",
            r"\1\2",
            out
        )
    return out
            

def medium_compact(soup):
    output = prettify(soup)
    output = re.sub(r"<\?xml.*?\?>", "", output).strip()
    return output

def high_compact(soup):
    """极简表格压缩方案"""
    output = prettify(soup, compact = True)
    output = re.sub(r"<\?xml.*?\?>", "", output)
    output = r"<!-- 标签映射表={<table>:<t>,<tr>:<r>,<td>:<d>,<th>:<h>} -->" + output
    return output

def simplify_html_table(html_content, compact_level = 0):
    """主函数：简化XHTML表格内容"""
    soup = BeautifulSoup(html_content, 'lxml')

    # 处理
    remove_unnecessary_elements(soup)
    remove_style_attributes(soup)
    flatten_paragraphs(soup,"font")
    flatten_paragraphs(soup,"b")
    remove_empty_rows(soup)
    
    for table in soup.find_all('table'):
        remove_trailing_empty_cells(table)
    
    add_basic_table_styles(soup)

    output = ""
    if compact_level == 0:
        output = re.sub(r'\s+', ' ', str(soup))
    elif compact_level == 1:
        output = medium_compact(get_first_table(soup))
    else:
        output = high_compact(get_first_table(soup))
    
    return output


with open('./xls/沪_13_财务支出审批及报销流程简表.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

simplified_html = simplify_html_table(html_content, compact_level=2)

with open('./xls/output2.html', 'w', encoding='utf-8') as f:
    f.write(simplified_html)

In [4]:

from pathlib import Path
for in_path in Path("./xls").glob("*.html"):
    with open(in_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    simplified_html = simplify_html_table(html_content, in_browser=True, compact=False)
    
    with open(in_path, 'w', encoding='utf-8') as f:
        f.write(simplified_html)